In [1]:
import torch

In [121]:
from torch_geometric_temporal.signal import temporal_signal_split
from torch_geometric_temporal.signal import StaticGraphTemporalSignal
import csv
import numpy as np

lags = 6
edges = np.array([[1, 0]]).T
edge_weights = np.ones(edges.shape[0])

data_array = []
with open('data.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    next(csv_reader)
    for row in csv_reader:
        data_row = [float(value) for value in row[1:-1]]
        data_array.append(data_row)

data_2d_array = np.array(data_array)
features = [
    data_2d_array[i : i + lags, :].T
    for i in range(data_2d_array.shape[0] - lags)
]
targets = [
    data_2d_array[i + lags, :].T
    for i in range(data_2d_array.shape[0] - lags)
]

dataset = StaticGraphTemporalSignal(edges, edge_weights, features, targets)

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.2)


In [122]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import DCRNN

class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features):
        super(RecurrentGCN, self).__init__()
        print(node_features)
        self.recurrent = DCRNN(node_features, 32, 1)
        self.linear = torch.nn.Linear(32, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

In [123]:
from tqdm import tqdm

model = RecurrentGCN(node_features = 6)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

for epoch in tqdm(range(200)):
    cost = 0
    for time, snapshot in enumerate(train_dataset):
        y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
        cost = cost + torch.mean((y_hat-snapshot.y)**2)
    cost = cost / (time+1)
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

6


100%|██████████| 200/200 [00:08<00:00, 23.01it/s]


In [124]:
model.eval()
cost = 0

for time, snapshot in enumerate(test_dataset):
    # print(snapshot.x)
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    print(y_hat)
    cost = cost + torch.mean((y_hat-snapshot.y)**2)
cost = cost / (time+1)
cost = cost.item()
print("MSE: {:.4f}".format(cost))

tensor([[30.2707],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
        [30.3066]], grad_fn=<AddmmBackward0>)
tensor([[30.3066],
      